In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from torch.utils.data import DataLoader, TensorDataset

from utility_functions import gen_data_dict, get_data, SquaredHingeLoss

np.random.seed(4)
torch.manual_seed(4)

In [2]:
# chosen feature for training
chosen_feature = ['std_deviation', 'count', 'sum_diff', 'range_value', 'abs_skewness']

# hyper para in cv
cv_batch_size = 4
cv_n_folds    = 2
cv_n_ites     = 200

# batch size in training
train_batch_size = 1

# learning rate in cv and training
lr = 0.001

# hyper parameters to try
n_hiddens_values  = [1, 2, 3]
layer_size_values = [4, 8, 16]

configs = [{'n_hiddens': 0, 'layer_size': 0}]
configs += [{'n_hiddens': n, 'layer_size': s} for n in n_hiddens_values for s in layer_size_values]

# getting dataframe of error count
err_fold1_df = pd.read_csv('../1_training_data/errors_fold1_base_10.csv')
err_fold2_df = pd.read_csv('../1_training_data/errors_fold2_base_10.csv')

# getting sequences and labels data
seqs   = gen_data_dict('../0_sequences_labels/signals.gz')
labels = gen_data_dict('../0_sequences_labels/labels.gz')

In [3]:
# Chose features
features = pd.read_csv('../1_training_data/seq_features.csv').iloc[:, 1:][chosen_feature].to_numpy()

# normalize them
mean = np.mean(features, axis=0)
std_dev = np.std(features, axis=0)
features = (features-mean)/std_dev

# convert to torch tensor
features = torch.Tensor(features)

# verify feature input size
feature_input_size = len(chosen_feature)
print("Number of features:", feature_input_size)

# verify the shape
print("Shape of features tensor:", list(features.shape))

Number of features: 5
Shape of features tensor: [413, 5]


In [4]:
# target intervals
target_df_1 = pd.read_csv('../1_training_data/target_lambda_fold1_base_10.csv')
target_df_2 = pd.read_csv('../1_training_data/target_lambda_fold2_base_10.csv')

targets_low_1  = torch.Tensor(target_df_1.iloc[:, 1:2].to_numpy())
targets_high_1 = torch.Tensor(target_df_1.iloc[:, 2:3].to_numpy())
targets_low_2  = torch.Tensor(target_df_2.iloc[:, 1:2].to_numpy())
targets_high_2 = torch.Tensor(target_df_2.iloc[:, 2:3].to_numpy())

target_fold1 = torch.cat((targets_low_1, targets_high_1), dim=1)
target_fold2 = torch.cat((targets_low_2, targets_high_2), dim=1)

In [5]:
# Model architect
class DLModel(nn.Module):
    def __init__(self, input_size, hidden_layers, hidden_size):
        super(DLModel, self).__init__()
        self.input_size    = input_size
        self.hidden_layers = hidden_layers
        self.hidden_size   = hidden_size

        if(self.hidden_layers == 0):
            self.linear_model = nn.Linear(input_size, 1)                                                        # Define linear model
        else:
            self.input_layer = nn.Linear(input_size, hidden_size)                                               # Define input layer
            self.hidden = nn.ModuleList([nn.Linear(hidden_size, hidden_size) for _ in range(hidden_layers-1)])  # Define hidden layers
            self.output_layer = nn.Linear(hidden_size, 1)                                                       # Define output layer
        
    def forward(self, x):
        if(self.hidden_layers == 0):
            return self.linear_model(x)
        else:
            x = torch.relu(self.input_layer(x))
            for layer in self.hidden:
                x = torch.relu(layer(x))
            x = self.output_layer(x)
            return x

In [6]:
def cv_learn(n_splits, X, y, n_hiddens, layer_size, lr, n_ite):
    
    # Define the number of folds for cross-validation
    kf = KFold(n_splits, shuffle=True, random_state=0)

    # loss function
    loss_func = SquaredHingeLoss(margin=1)

    # learn best ite
    total_train_losses = np.zeros(n_ite)
    total_val_losses   = np.zeros(n_ite)
    for train_index, val_index in kf.split(X):

        # Split the data into training and validation sets
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]

        # Create DataLoader
        dataset    = TensorDataset(X_train, y_train)
        dataloader = DataLoader(dataset, batch_size=cv_batch_size, shuffle=True)

        # Define your model
        model = DLModel(feature_input_size, n_hiddens, layer_size)

        # define optimizer
        optimizer = optim.Adam(model.parameters(), lr)

        # Training loop for the specified number of iterations
        train_losses = []
        val_losses   = []
        for i in range(n_ite):
            # training
            train_loss = 0
            for inputs, labels in dataloader:
                optimizer.zero_grad()
                loss = loss_func(model(inputs), labels)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()

            # validating
            model.eval()
            with torch.no_grad():
                val_loss = loss_func(model(X_val), y_val)

            # add train_loss and val_loss into arrays
            train_losses.append(train_loss/len(dataloader))
            val_losses.append(val_loss.item())

        total_train_losses += train_losses
        total_val_losses += val_losses

    best_no_ite = np.argmin(total_val_losses)
    return best_no_ite + 1

In [7]:
# train model
def train_model(X, y, n_hiddens, layer_size, lr, n_ites):
    model = DLModel(feature_input_size, n_hiddens, layer_size)
    loss_func = SquaredHingeLoss(margin=1)
    optimizer = optim.Adam(model.parameters(), lr)

    # Create DataLoader
    dataset    = TensorDataset(X, y)
    dataloader = DataLoader(dataset, batch_size=train_batch_size, shuffle=True)

    # Training loop
    for _ in range(n_ites):
        for inputs, labels in dataloader:
            optimizer.zero_grad()
            loss = loss_func(model(inputs), labels)
            loss.backward()
            optimizer.step()
    
    return model

In [8]:
def get_df_stat(ldas1, ldas2, err_fold1_df, err_fold2_df, seqs, labels):
    header = ['sequenceID', 'lda_fold1', 'lda_fold2', 'fold_1_total_labels', 'fold_2_total_labels', 'fold1_err', 'fold2_err']
    rows = []
    for i in range(len(seqs)):
        # get total labels
        _, neg_start_1, _, pos_start_1, _, neg_start_2, _, pos_start_2, _ = get_data(i, seqs, labels)
        fold1_total_labels = len(neg_start_1) + len(pos_start_1)
        fold2_total_labels = len(neg_start_2) + len(pos_start_2)

        # round lambda
        ldas1 = [round(num*2)/2 for num in ldas1]
        ldas2 = [round(num*2)/2 for num in ldas2]

        # get err
        fold1_err = err_fold1_df.iloc[i][str(ldas1[i])]
        fold2_err = err_fold2_df.iloc[i][str(ldas2[i])]
        
        # add row to rows
        row = [seqs[i][0], ldas1[i], ldas2[i], fold1_total_labels, fold2_total_labels, fold1_err, fold2_err]
        rows.append(row)
    
    df = pd.DataFrame(rows, columns=header)
    return df

In [9]:
def try_model(full_X, X1, X2, y1, y2, config, err_fold1_df, err_fold2_df, seqs, labels):
    n_hiddens  = config['n_hiddens']
    layer_size = config['layer_size']

    best_no_ite_1 = cv_learn(cv_n_folds, X1, y1, n_hiddens, layer_size, lr, cv_n_ites)
    best_no_ite_2 = cv_learn(cv_n_folds, X2, y2, n_hiddens, layer_size, lr, cv_n_ites)

    model1 = train_model(X1, y1, n_hiddens, layer_size, lr, best_no_ite_1)
    model2 = train_model(X2, y2, n_hiddens, layer_size, lr, best_no_ite_2)

    with torch.no_grad():
        ldas1 = model1(full_X).numpy().reshape(-1)
        ldas2 = model2(full_X).numpy().reshape(-1)

    df = get_df_stat(ldas1, ldas2, err_fold1_df, err_fold2_df, seqs, labels)
    return df

In [10]:
# train all models parallel and get stats dfs
torch.manual_seed(0)
dfs = Parallel(n_jobs=len(configs), verbose=1)(delayed(try_model)(features, features, features, target_fold1, target_fold2,
                                                        configs[i], err_fold1_df, err_fold2_df, seqs, labels) for i in range(len(configs)))

[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   2 out of  10 | elapsed:  2.1min remaining:  8.5min
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:  4.6min finished


In [11]:
# printing results
results = []
for i in range(len(dfs)):
    df = dfs[i]
    
    total_label_fold1 = df['fold_1_total_labels'].sum()
    total_label_fold2 = df['fold_2_total_labels'].sum()
    err1 = df['fold1_err'].sum()
    err2 = df['fold2_err'].sum()
    rate1 = (total_label_fold1 - err1)/total_label_fold1
    rate2 = (total_label_fold2 - err2)/total_label_fold2

    results.append({
        'n_hiddens': configs[i]['n_hiddens'],
        'layer_size': configs[i]['layer_size'],
        'fold1_test': rate1 * 100,
        'fold2_test': rate2 * 100
    })

df_results = pd.DataFrame(results)
print(df_results)

   n_hiddens  layer_size  fold1_test  fold2_test
0          0           0   78.457447   81.538462
1          1           4   78.856383   79.807692
2          1           8   78.590426   79.615385
3          1          16   79.388298   81.153846
4          2           4   78.457447   80.192308
5          2           8   78.856383   79.615385
6          2          16   79.388298   80.769231
7          3           4   79.255319   79.807692
8          3           8   79.787234   77.884615
9          3          16   81.515957   78.269231


In [12]:
# Calculate the average of fold1_test and fold2_test
fold1_test_avg = df_results['fold1_test'].mean()
fold2_test_avg = df_results['fold2_test'].mean()

print("Average fold1_test: %.2f" % fold1_test_avg)
print("Average fold2_test: %.2f" % fold2_test_avg)

Average fold1_test: 79.26
Average fold2_test: 79.87
